In [ ]:
!nvidia-smi

/bin/bash: nvidia-smi: command not found


In [ ]:
# This get the RAPIDS-Colab install files and test check your GPU.  Run this and the next cell only.
# Please read the output of this cell.  If your Colab Instance is not RAPIDS compatible, it will warn you and give you remediation steps.
"""
try:
  import cuml
except (ImportError, KeyError, ModuleNotFoundError):
  !git clone https://github.com/rapidsai/rapidsai-csp-utils.git
  !python rapidsai-csp-utils/colab/pip-install.py
"""

'\ntry:\n  import cuml\nexcept (ImportError, KeyError, ModuleNotFoundError):\n  !git clone https://github.com/rapidsai/rapidsai-csp-utils.git\n  !python rapidsai-csp-utils/colab/pip-install.py\n'

In [ ]:
# check if cuda is installed correctly
#!nvcc --version

In [ ]:
# Mount google drive to colab and change to correct directory
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/Othercomputers/ThinkPad/master-thesis-vt23

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/Othercomputers/ThinkPad/master-thesis-vt23


In [ ]:
# Try to import packages, if exception is thrown install dependencies and kill runtime
%cd /content/drive/Othercomputers/ThinkPad/master-thesis-vt23
try:
  import sdmetrics
  from pycaret.classification import *
except (ImportError, KeyError, ModuleNotFoundError):
  ## code to install dependencies
  !pip install sdv
  !pip install sdmetrics
  !pip install kmodes
  !pip install pomegranate==0.14.8   #needed for sdmetrics BNLikelihood 
  !pip install pycaret==3.0.0
  display('Stopping RUNTIME! Colaboratory will restart automatically. Please run again.')
  import os
  os.kill(os.getpid(), 9)

/content/drive/Othercomputers/ThinkPad/master-thesis-vt23


In [ ]:
%cd /content/drive/Othercomputers/ThinkPad/master-thesis-vt23/notebooks

import pandas as pd 
import sys
import os
import re

from sdv.metadata import SingleTableMetadata

sys.path.append('../src')
from PF_metrics import compute_all_pf_measures
from utils import (getExperimentConfig, 
                   getPicklesFromDir, 
                   get_synthetic_filepaths_from_original_data_id)

config = getExperimentConfig()
folders = config['folders']

settings = getPicklesFromDir(folders['settings_dir'])

/content/drive/Othercomputers/ThinkPad/master-thesis-vt23/notebooks


Run pf measures for MNIST dataset

In [ ]:
data_id = 'D305'
run_quality = 'Q2'
file_path = f"../data/result/{data_id}{run_quality}_pf_measures.csv"


if os.path.exists(file_path):
    result_df = pd.read_csv(file_path)
else:
    result_df = pd.DataFrame(columns=['DatasetName'])

for dataset_setting in settings:
    #Skip specified dataset
    if data_id != dataset_setting['meta']['id']:
      continue

    original_data = pd.read_csv(folders['real_dir']+dataset_setting['meta']['filename'], dtype=dataset_setting['meta']['cols_dtype'])
    metadata = SingleTableMetadata().load_from_json(dataset_setting['meta']['meta_filepath']).to_dict()
    original_data_id = dataset_setting['meta']['id']
    synthetic_datasets = get_synthetic_filepaths_from_original_data_id(original_data_id)
    
    print(f"Data id: {original_data_id}")
    for sd_filename in synthetic_datasets:   
        sd_id = os.path.splitext(sd_filename)[0]
        quality = re.findall('Q\d+', sd_id)[0]
        sd_path = folders['sd_dir']+sd_filename

        if run_quality != quality:
          continue
        
        # Check if the dataset has already been evaluated
        row_exists = ((result_df['DatasetName'] == sd_id)).any()
        if not row_exists:
            print(f"Computing PF on: {sd_id}")
            synthetic_data = pd.read_csv(sd_path)

            pf_measures = compute_all_pf_measures(original_data=original_data,
                                                  synthetic_data=synthetic_data,
                                                  dataset_meta=dataset_setting['meta'],
                                                  metadata=metadata,
                                                  SD_id=sd_id)
            pf_measures['Quality'] = quality
            pf_measures['Original dataset'] = original_data_id
            result_df = pd.concat([result_df, pf_measures], axis=0, ignore_index=True)

        # save the results
        result_df.to_csv(file_path, index=False)

Data id: D305
Computing PF on: SD305Q2_0
Cluster group: G_0.5, with 18 clusters
Value: 59.83367568467739, Time: 937.7192369929999
Cluster group: G_1, with 35 clusters
Value: 37.18631798070338, Time: 1483.0362643890003
Cluster group: G_1.5, with 52 clusters
Value: 26.82514844944871, Time: 1852.2658055619995
Cluster group: G_2, with 70 clusters
Value: 20.139903031092274, Time: 2546.532577390999
Computing PF on: SD305Q2_1
Cluster group: G_0.5, with 18 clusters
Value: 55.73470863673344, Time: 1071.64060006
Cluster group: G_1, with 35 clusters
Value: 38.361622938729845, Time: 1477.0481102230005
Cluster group: G_1.5, with 52 clusters
Value: 26.52178201267956, Time: 1591.9001611019994
Cluster group: G_2, with 70 clusters
Value: 21.067401579240908, Time: 2566.265312159001
Computing PF on: SD305Q2_2
Cluster group: G_0.5, with 18 clusters
Value: 66.93273467415123, Time: 880.0842888109983
Cluster group: G_1, with 35 clusters
Value: 39.56457458619745, Time: 1512.3198324390032
Cluster group: G_1.5,

Streaming output truncated to the last 5000 lines.
  File "<ipython-input-7-238329756a61>", line 36, in <cell line: 11>
    pf_measures = compute_all_pf_measures(original_data=original_data,
  File "/content/drive/Othercomputers/ThinkPad/master-thesis-vt23/notebooks/../src/PF_metrics.py", line 643, in compute_all_pf_measures
    result = ContinousKLDivergence_metric(o_data, s_data, metadata)
  File "/content/drive/Othercomputers/ThinkPad/master-thesis-vt23/notebooks/../src/utils.py", line 90, in timefunction_wrapper
    score = func(*args, **kwargs)
  File "/content/drive/Othercomputers/ThinkPad/master-thesis-vt23/notebooks/../src/PF_metrics.py", line 492, in ContinousKLDivergence_metric
    return ContinuousKLDivergence.compute(
  File "/usr/local/lib/python3.10/dist-packages/sdmetrics/single_table/multi_column_pairs.py", line 98, in compute
    return cls._compute(cls, real_data, synthetic_data, metadata, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sdmetrics/single_tabl

In [ ]:
#%cd /content/drive/Othercomputers/ThinkPad/master-thesis-vt23/notebooks
#%run -t Step2-Model-CPU.ipynb

In [ ]:
#%cd /content/drive/Othercomputers/ThinkPad/master-thesis-vt23/notebooks
#%run -t Step3-SDG.ipynb

In [ ]:
#%cd /content/drive/Othercomputers/ThinkPad/master-thesis-vt23/notebooks
#%run -t Step4-SD-Models-CPU.ipynb

In [ ]:
#%cd /content/drive/Othercomputers/ThinkPad/master-thesis-vt23/notebooks
#%run -t Step5-Measures_PF.ipynb